## Applied Data Analysis Project
Team: ToeStewBrr - Alexander Sternfeld, Marguerite Thery, Antoine Bonnet, Hugo Bordereaux

Dataset: CMU Movie Summary Corpus

### 1. Preprocessing data

In [8]:
import requests
import tarfile
import os
import pandas as pd
import seaborn as sns
import matplotlib as plt

Temporary configuration: Until we figure out how to extract the tar file directly from the url, please download the file and place it in the Data directory of this project. 

We first extract all files from the MoviesSummaries tar file. 

In [9]:
if not os.path.exists('Data/MovieSummaries'): 
    my_tar = tarfile.open('Data/MovieSummaries.tar') # Can't upload voluminous data to Github
    my_tar.extractall('./Data') # specify which folder to extract to
    my_tar.close()

We then explore the structure of each of the data files. 

### 1. plot_summaries.txt [29 M] 

Plot summaries of 42,306 movies extracted from the November 2, 2012 dump of English-language Wikipedia.  Each line contains the Wikipedia movie ID (which indexes into movie.metadata.tsv) followed by the summary.

In [10]:
plot_path = 'Data/MovieSummaries/plot_summaries.txt'
plot_cols = ['Wikipedia ID', 'Summary']
plot_df = pd.read_csv(plot_path, sep='\t', header=None, names=plot_cols, index_col=0)
plot_df


,Summary
Wikipedia ID,
23890098,"Shlykov, a hard-working taxi driver and Lyosha..."
31186339,The nation of Panem consists of a wealthy Capi...
20663735,Poovalli Induchoodan is sentenced for six yea...
2231378,"The Lemon Drop Kid , a New York City swindler,..."
595909,Seventh-day Adventist Church pastor Michael Ch...
...,...
34808485,"The story is about Reema , a young Muslim scho..."
1096473,"In 1928 Hollywood, director Leo Andreyev look..."
35102018,American Luthier focuses on Randy Parsons’ tra...


Trying to extract "love words" from the summary

In [25]:
#Returns 1 if a word from a dictionnary is in the observed list
def num_words_in_list(x, key, dictionary):
  count = 0
  for word in x.split(" "): 
    if word.lower() in dictionary.get(key):
      count+=1
  return count

In [26]:
#Read set of words provided
love_actions = pd.read_table('Data/love_list.txt', skiprows= 8, nrows = 25, names=['Love_Actions'], encoding = "ISO-8859-1", dtype=str)
love_synonyms = pd.read_table('Data/love_list.txt', skiprows= 36, names=['Love_Synonyms'], encoding = "ISO-8859-1", dtype=str)

#Create corresponding pandas series
love_actions = love_actions['Love_Actions'].squeeze()
love_synonyms = love_synonyms['Love_Synonyms'].squeeze()

#Convert them to lists
love_actions = love_actions.tolist()
love_synonyms = love_synonyms.tolist()

#Create dictionnary of love
love_dict = dict([
    ('# of Love Actions', love_actions),
    ('# of Love Synonyms', love_synonyms)
])

#Counts the occurences of love actions and synonyms
for category in love_dict.keys():
  plot_df[category] = plot_df['Summary'].apply(lambda x: num_words_in_list(x,category,love_dict))

In [34]:
plot_df.head()
plot_df.sort_values(by = '# of Love Actions', axis = 0, ascending = False)

,Summary,# of Love Actions,# of Love Synonyms
Wikipedia ID,,,
6283620,The drama starts off about a young woman name ...,85,0
8950238,{{Plot}} Takagi and Sato chase thieves that ro...,27,0
948608,"In the summer of the 1957, Dani is a 14-year-...",25,0
7487962,Treasure Lee learns that her biological mothe...,23,0
3845313,Hye-young is an artist who makes her living b...,17,0
...,...,...,...
1855828,The film is about a young man experiencing the...,0,0
7302414,"At the end of a raid on a cheese factory, Hubi...",0,0
33569921,"As students Fritz , Ulrich and Johannes Wolf ...",0,0


### 2. movie.metadata.tsv.gz [3.4 M]


Metadata for 81,741 movies, extracted from the Noverber 4, 2012 dump of Freebase.  Tab-separated; columns:

1. Wikipedia movie ID
2. Freebase movie ID
3. Movie name
4. Movie release date
5. Movie box office revenue
6. Movie runtime
7. Movie languages (Freebase ID:name tuples)
8. Movie countries (Freebase ID:name tuples)
9. Movie genres (Freebase ID:name tuples)


In [4]:
movie_path = 'Data/MovieSummaries/movie.metadata.tsv'
movie_cols = ['Wikipedia ID', 'Freebase ID', 'Name', 'Release date', 
              'Box office revenue', 'Runtime', 'Languages', 'Countries', 'Genres']
movie_df = pd.read_csv(movie_path, sep='\t', header=None, names=movie_cols, index_col=0)
movie_df



,Freebase ID,Name,Release date,Box office revenue,Runtime,Languages,Countries,Genres
Wikipedia ID,,,,,,,,
975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"
...,...,...,...,...,...,...,...,...
35228177,/m/0j7hxnt,Mermaids: The Body Found,2011-03-19,NaN,120.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/07s9rl0"": ""Drama""}"
34980460,/m/0g4pl34,Knuckle,2011-01-21,NaN,96.0,"{""/m/02h40lc"": ""English Language""}","{""/m/03rt9"": ""Ireland"", ""/m/07ssc"": ""United Ki...","{""/m/03bxz7"": ""Biographical film"", ""/m/07s9rl0..."
9971909,/m/02pygw1,Another Nice Mess,1972-09-22,NaN,66.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06nbt"": ""Satire"", ""/m/01z4y"": ""Comedy""}"


### 3. character.metadata.tsv.gz [14 M]

Metadata for 450,669 characters aligned to the movies above, extracted from the Noverber 4, 2012 dump of Freebase.  Tab-separated; columns:

1. Wikipedia movie ID
2. Freebase movie ID
3. Movie release date
4. Character name
5. Actor date of birth
6. Actor gender
7. Actor height (in meters)
8. Actor ethnicity (Freebase ID)
9. Actor name
10. Actor age at movie release
11. Freebase character/actor map ID
12. Freebase character ID
13. Freebase actor ID


In [5]:
char_path = 'Data/MovieSummaries/character.metadata.tsv'
char_cols = ['Wikipedia ID', 'Freebase ID', 'Release date', 'Character name', 'Date of birth', 
             'Gender', 'Height', 'Ethnicity', 'Actor name', 'Actor age at release', 
             'Freebase character/map ID', 'Freebase character ID', 'Freebase actor ID']
char_df = pd.read_csv(char_path, sep='\t', header=None, names=char_cols, index_col=0)
char_df

,Freebase ID,Release date,Character name,Date of birth,Gender,Height,Ethnicity,Actor name,Actor age at release,Freebase character/map ID,Freebase character ID,Freebase actor ID
Wikipedia ID,,,,,,,,,,,,
975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.620,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4
975900,/m/03vyhn,2001-08-24,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l
975900,/m/03vyhn,2001-08-24,Sgt Jericho Butler,1967-09-12,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/0bgchnq,/m/034hyc
975900,/m/03vyhn,2001-08-24,Bashira Kincaid,1977-09-25,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/0bgchp9,/m/01y9xg
...,...,...,...,...,...,...,...,...,...,...,...,...
913762,/m/03pcrp,1992-05-21,Elensh,1970-05,F,NaN,NaN,Dorothy Elias-Fahn,NaN,/m/0kr406c,/m/0kr406h,/m/0b_vcv
913762,/m/03pcrp,1992-05-21,Hibiki,1965-04-12,M,NaN,NaN,Jonathan Fahn,27.0,/m/0kr405_,/m/0kr4090,/m/0bx7_j
28308153,/m/0cp05t9,1957,NaN,1941-11-18,M,1.730,/m/02w7gg,David Hemmings,15.0,/m/0g8ngmc,NaN,/m/022g44


### 4. corenlp_plot_summaries.tar.gz [628 M, separate download]

The plot summaries from above, run through the Stanford CoreNLP pipeline (tagging, parsing, NER and coref). Each filename begins with the Wikipedia movie ID (which indexes into movie.metadata.tsv).


In [6]:
#NER tag = person can give us the characters mention in the plot summary. 


In [7]:
import gzip
import xml.etree.ElementTree as ET
#I uploaded the corenlp_plot_summaries from local
directory = './corenlp_plot_summaries'

#TODO: Need to extract each file and convert them to xml 
#For loop to open every file in the directory
for filename in os.listdir(directory):
  f = os.path.join(directory, filename) 
  if os.path.isfile(f):
    #open and store file as xml 


IndentationError: expected an indented block (3445337635.py, line 11)

In [ ]:
#Use file I already extracted on my computer to run some tests
tree = ET.parse('3217.xml')
root = tree.getroot()

print(len(root.findall('.//*governor'))) #use parse or basic-dependencies to have more info 
#print(root.findall('.//*governor').text())
for l in root.findall('.//*NER'): 
  if len(l.text) > 1:
    print(l.text)
